## Installing lightfm & Import libraries

In [0]:
!pip install -q lightfm

In [0]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score

## Fetch data from movielens dataset

In [0]:
#fetch data and format it
data = fetch_movielens(min_rating=4.0)

In [14]:
for col in data:
  print (col)

train
test
item_features
item_feature_labels
item_labels


In [15]:
#print training and testing data
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


## Modelling
Here we are using three types of loss function
- logistic: useful when both positive (1) and negative (-1) interactions are present.
- bpr : Bayesian Personalised Ranking pairwise loss.
- warp : Weighted Approximate-Rank Pairwise loss. 

In [16]:
#create model
model = LightFM(loss='warp')
model1 = LightFM(loss='logistic')
model2 = LightFM(loss='bpr')
#train model
model.fit(data['train'], epochs=30, num_threads=4)
model1.fit(data['train'], epochs=30, num_threads=4)
model2.fit(data['train'], epochs=30, num_threads=4)

In [19]:
#number of users and movies in training data
n_users, n_items = data['train'].shape

# Indices of movie rated by the user_id '3'
#data['train'].todense()[3].nonzero()
data['train'].tocsr()[3].indices

array([ 10, 257, 270, 299, 300, 323, 326, 328, 358, 359, 361], dtype=int32)

In [0]:
data['item_labels'][data['train'].tocsr()[3].indices]

array(['Seven (Se7en) (1995)', 'Contact (1997)',
       'Starship Troopers (1997)', 'Air Force One (1997)',
       'In & Out (1997)', 'Lost Highway (1997)', 'Cop Land (1997)',
       'Desperate Measures (1998)', 'Assignment, The (1997)',
       'Wonderland (1997)', 'Blues Brothers 2000 (1998)'], dtype=object)

In [0]:
data['item_labels'].shape

(1682,)

## Check Metrics Score
Here we are using AUC Score as a metrics, where 1 being the best model.

In [0]:
auc = auc_score(model, data['test'],train_interactions=data['train'])
auc1 = auc_score(model1, data['test'],train_interactions=data['train'])
auc2 = auc_score(model2, data['test'],train_interactions=data['train'])
print ('WARP AUC Score: {} '.format(auc.sum()/auc.shape))
print ('Logistic AUC Score : {} '.format(auc1.sum()/auc1.shape))
print ('BPR AUC Score : {} '.format(auc2.sum()/auc2.shape))

AUC : [0.93351599] 
AUC1 : [0.87956248] 
AUC2 : [0.86359753] 


Since, Warp model outperforms the other two model. We will be using warp model to predict the movies.

In [0]:
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies they already like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        #rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [0]:
sample_recommendation(model, data, [3, 25, 450])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        Scream (1996)
        Air Force One (1997)
        Conspiracy Theory (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        L.A. Confidential (1997)
        Contact (1997)
        Fargo (1996)
User 450
     Known positives:
        Contact (1997)
        George of the Jungle (1997)
        Event Horizon (1997)
     Recommended:
        Kiss the Girls (1997)
        Conspiracy Theory (1997)
        Seven Years in Tibet (1997)
